# VisioNova: DeBERTa-v3 AI Text Detection Training

This notebook trains the `microsoft/deberta-v3-base` model using the Google Colab runtime connected via VS Code.

**Important:**
- This notebook runs on the remote Colab instance.
- Models are saved to the remote filesystem.
- You must manually download the trained model folder from the VS Code file explorer after training.

In [ ]:
# 1. SETUP & DEPENDENCIES
print("Installing dependencies...")
!pip install -q transformers datasets accelerate scikit-learn sentencepiece

In [ ]:
# 2. IMPORTS & CONFIGURATION
import os
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# STRICT GPU CHECK
if not torch.cuda.is_available():
    raise RuntimeError(
        "\n\n🛑 GPU NOT DETECTED! 🛑\n"
        "Make sure you are connected to a Colab Runtime with GPU enabled (Runtime > Change runtime type > GPU)."
    )
else:
    print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")

# --- CONFIG ---
DATASET_NAME = "artem9k/ai-text-detection-pile"
MODEL_ID = "microsoft/deberta-v3-base"
SAVE_DIR_NAME = f"{DATASET_NAME.split('/')[-1]}_DeBERTa_v3"
OUTPUT_DIR = os.path.join("./VisioNova_Models", SAVE_DIR_NAME)

# --- HYPERPARAMETERS ---
EPOCHS = 3
BATCH_SIZE = 8
LEARNING_RATE = 2e-5
MAX_LEN = 512

print(f"Model will be saved to: {OUTPUT_DIR}")

In [ ]:
# 3. LOAD & PREPARE DATASET
print(f"Loading dataset: {DATASET_NAME}...")
dataset = load_dataset(DATASET_NAME)

# Print dataset info for debugging
print(f"Dataset splits: {list(dataset.keys())}")
sample_split = list(dataset.keys())[0]
print(f"Columns in '{sample_split}': {dataset[sample_split].column_names}")

if 'train' in dataset:
    split_dataset = dataset['train'].train_test_split(test_size=0.1, seed=42)
else:
    split_dataset = dataset.train_test_split(test_size=0.1, seed=42)

print(f"Training Samples: {len(split_dataset['train'])}")
print(f"Validation Samples: {len(split_dataset['test'])}")

In [ ]:
# 4. CREATE LABELS FROM 'source' COLUMN
# The dataset has 'source' column indicating origin (e.g., 'human', 'chatgpt', 'gpt-4').
# We derive binary labels: 0 = Human, 1 = AI-generated.

# First, let's see unique sources
unique_sources = set(split_dataset['train']['source'][:10000])  # Sample first 10k
print(f"Sample sources found: {unique_sources}")

# Define human sources (add more if needed based on output above)
HUMAN_SOURCES = {'human', 'Human', 'wikipedia', 'reddit', 'news', 'books'}

def add_labels(example):
    """Add binary label based on source column."""
    source = example.get('source', '').lower()
    # If source contains any human indicator, label as 0 (human)
    is_human = any(h.lower() in source for h in HUMAN_SOURCES)
    example['labels'] = 0 if is_human else 1
    return example

print("Adding labels based on 'source' column...")
split_dataset = split_dataset.map(add_labels)

# Verify label distribution
train_labels = split_dataset['train']['labels']
human_count = sum(1 for l in train_labels if l == 0)
ai_count = sum(1 for l in train_labels if l == 1)
print(f"Label distribution - Human: {human_count}, AI: {ai_count}")

In [ ]:
# 5. TOKENIZATION
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def tokenize_function(examples):
    return tokenizer(
        examples["text"], 
        padding="max_length", 
        truncation=True, 
        max_length=MAX_LEN
    )

print("Tokenizing dataset...")
tokenized_datasets = split_dataset.map(tokenize_function, batched=True)

# Verify columns
print(f"Columns after tokenization: {tokenized_datasets['train'].column_names}")

# Set format for PyTorch - only include columns that exist
required_cols = ["input_ids", "attention_mask", "labels"]
# token_type_ids may not exist for all models
if "token_type_ids" in tokenized_datasets['train'].column_names:
    required_cols.insert(1, "token_type_ids")

tokenized_datasets.set_format(type="torch", columns=required_cols)
print(f"Format set with columns: {required_cols}")

In [ ]:
# 6. MODEL & TRAINER SETUP
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID, 
    num_labels=2,
    id2label={0: "HUMAN", 1: "AI"},
    label2id={"HUMAN": 0, "AI": 1}
)

training_args = TrainingArguments(
    output_dir="./checkpoints",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True, 
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)
print("Trainer ready!")

In [ ]:
# 7. TRAIN & SAVE
print("Starting training...")
trainer.train()

print(f"\nSaving model to: {OUTPUT_DIR}")
os.makedirs(OUTPUT_DIR, exist_ok=True)
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# Save metrics
metrics = trainer.evaluate()
print(f"Final Metrics: {metrics}")
with open(os.path.join(OUTPUT_DIR, "metrics.txt"), "w") as f:
    f.write(str(metrics))

print("\n✅ Training Complete!")
print(f"The model is saved at: {os.path.abspath(OUTPUT_DIR)}")
print("IMPORTANT: Download the folder from VS Code File Explorer to your local machine.")

### Post-Training Instructions
Once you have downloaded the folder, place it in your local project at `backend/text_detector/model/`.